<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/09%20-%20Word%20embedding%20e%20Word2Vec/glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GloVe
Il modello **GloVe (Global Vectors for Word Representation)** è un metodo non supervisionato per l'apprendimento della rappresentazione vettoriale delle parole. La prima versione di GloVe è stata creata nei laboratori di NLP di Standford nel 2014, i quali hanno messo a disposizione il modello pre-addestrato sull'intero corpus di testo di Wikipedia, con qualcosa come 6 miliardi di parole. In questo notebook utilizzeremo il modello pre-addestrato per creare lo strato di embedding di una rete neurale per classificare la polarità di recensioni di film utilizzando l'IMDB Movie Reviews Dataset.

## Prepariamo i dati

In precedenza abbiamo visto come scaricare e preprocessare l'IMDB Movie Reviews Dataset, Keras mette a disposizione tale dataset già preprocessato.
<br>**ATTENZIONE**<br>
Se caricando il dataset ottieni questo errore:<br>
*ValueError: Object arrays cannot be loaded when allow_pickle=False*
<br>
questo è casuato da un bug nell'ultima versione di keras, per correggerlo esegui il downgrade di numpy usando la cella di codice qui sotto e riavvia il kernel (su Colaboratory seleziona Runtime dalla barra dei comandi e clicca su Restart Runtime).

In [28]:
!pip install numpy==1.16.2

Requirement already up-to-date: keras in /usr/local/lib/python3.7/dist-packages (2.4.3)


In [189]:
import numpy as np
from tensorflow.keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data()

print(X_train[0][:10])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]


Ogni riga della lista con le features corrisponde ad una frase, ogni colonna contiene l'indice di una parola all'interno del vocabolario dell'intero corpus di testo. Il vettore con il target contiene un unico valore che può essere 0 per una recensione negativa o 1 per una recensione positiva.<br><br>

Proviamo a ricostruire la prima frase dagli indici, just for fun :)

In [190]:
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

In [191]:
index_word = {value:key for key,value in word_index.items()}
print(' '.join(index_word[id] for id in X_train[0] ))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sho

Per rendere le features un buon input per il nostro modello dobbiamo fare in modo che ogni frase abbia la stessa lunghezza, per farlo possiamo usare la funzione *pad_sequences(text)* di keras, che riduce tutte le frasi ad una lunghezza prefissata troncando quelle troppo lunghe e aggiungendo degli zeri a quelle troppo brevi. Usiamo una lunghezza comune di 50 parole.

In [192]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 50

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 50)

Adesso i dati sono pronti.

## Carichiamo il modello GloVe

Possiamo scaricare il modello pre-addestrato [da questo link](http://nlp.stanford.edu/data/glove.6B.zip). Se utilizzi Google Colab o comunque hai wget installato sul tuo computer esegui pure la cella di codice qui sotto per scaricare il dataset.

In [3]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-04-24 08:36:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-24 08:36:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-24 08:36:10--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Ed estraiamo il file.

In [4]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


Lo zip contiene 4 file differenti, ognuno dei quali con un numero di dimensioni differente, 50, 100, 200 e 300. Leggiamo la prima riga del file con 50 dimensioni per comprendere come questo è strutturato.

In [193]:
with open("glove.6B.50d.txt") as file:
  for line in file.readlines():
    print(line)
    break

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581



Come vedi ogni riga corrisponde all'embedding per una determianta parola, il primo elemento della riga è la parola, i restanti solo i valori degli embedding. Alla luce di ciò definiamo una funzione per caricare gli embedding all'iterno di un dizionario la cui chiave è la parola. Usiamo il file con 100 dimensioni.

In [194]:
def load_embedding(filename):
  
  with open(filename) as file:
    lines = file.readlines()

  embedding = dict()
  
  for line in lines:
    parts = line.split()
    embedding[parts[0]] = np.asarray(parts[1:])
    
  return embedding

raw_embedding = load_embedding('glove.6B.100d.txt')
raw_embedding["man"]

array(['0.37293', '0.38503', '0.71086', '-0.65911', '-0.0010128',
       '0.92715', '0.27615', '-0.056203', '-0.24294', '0.24632',
       '-0.18449', '0.31398', '0.48983', '0.09256', '0.32958', '0.15056',
       '0.57317', '-0.18529', '-0.52277', '0.46191', '0.92038',
       '0.031001', '-0.16246', '-0.40567', '0.78621', '0.57722',
       '-0.53501', '-0.68228', '0.16987', '0.3631', '-0.071773',
       '0.47233', '0.027806', '-0.14951', '0.17543', '-0.37573',
       '-0.78517', '0.58171', '0.86859', '0.031445', '-0.45897',
       '-0.040917', '0.95897', '-0.16975', '0.13045', '0.27434',
       '-0.069485', '0.022402', '0.24977', '-0.21536', '-0.32406',
       '-0.39867', '0.68613', '1.7923', '-0.37848', '-2.2477', '-0.77025',
       '0.46582', '1.2411', '0.57756', '0.41151', '0.84328', '-0.54259',
       '-0.16715', '0.73927', '-0.093477', '0.90278', '0.50889',
       '-0.50031', '0.26451', '0.15443', '-0.29432', '0.10906',
       '-0.26667', '0.35438', '0.049079', '0.18018', '-0.5859'

Adesso dobbiamo trasformare il dizionario in una matrice, in cui ogni riga rappresenta contiene la rappresentazione vettoriale della parola che si trova alla stessa posizione all'interno del vocabolario dell'intero corpus di testo. Keras ci mette a disposizione anche il vocabolario già creato per questo dataset. possiamo ottenerlo con la funzione *.get_word_index()*.

In [195]:
word_index = imdb.get_word_index()
word_index["man"]

129

Definiamo la funzione per creare la matrie ed utilizziamola.

In [196]:
def get_weight_matrix(embedding, word_index):
  
  vocab_size = len(word_index)+3
  
  weight_matrix = np.zeros((vocab_size, 100))

  for word, i in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
      weight_matrix[i] = vector
      
  return weight_matrix

embedding_vectors = get_weight_matrix(raw_embedding, word_index)
embedding_vectors.shape

(88587, 100)

# Creiamo la rete
Cominciamo creando lo strato di embedding della rete, i primi due parametri saranno il numero di parole e il numero di dimensioni, passiamo gli embeddings già calcolati all'interno del parametro *weights* e settiamo il parametro *trainable* a False per indicare di non modificare questo strato della rete durante l'addestramento.

In [203]:
from keras.layers import Embedding

embedding_layer = Embedding(embedding_vectors.shape[0], 100, weights=[embedding_vectors], trainable=False, input_length=maxlen)

Ora creiamo la nostra rete neurale aggiungendo al primo strato lo strato di embedding appena creato, usiamo la classe *Flatten* di Keras per spacchettare la matrice con il word embedding in un unico vettore che unisce le righe della matrice.

In [198]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Compiliamo il modello ed avviamo l'addestramento.

In [199]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, epochs=10)

Epoch 1/10
49/49 [==============================] - 1s 4ms/step - loss: 0.7344 - accuracy: 0.5169
Epoch 2/10
49/49 [==============================] - 0s 5ms/step - loss: 0.6757 - accuracy: 0.5826
Epoch 3/10
49/49 [==============================] - 0s 4ms/step - loss: 0.6491 - accuracy: 0.6200
Epoch 4/10
49/49 [==============================] - 0s 4ms/step - loss: 0.6386 - accuracy: 0.6335
Epoch 5/10
49/49 [==============================] - 0s 4ms/step - loss: 0.6244 - accuracy: 0.6484
Epoch 6/10
49/49 [==============================] - 0s 4ms/step - loss: 0.6170 - accuracy: 0.6607
Epoch 7/10
49/49 [==============================] - 0s 4ms/step - loss: 0.6092 - accuracy: 0.6641
Epoch 8/10
49/49 [==============================] - 0s 4ms/step - loss: 0.6027 - accuracy: 0.6746
Epoch 9/10
49/49 [==============================] - 0s 4ms/step - loss: 0.5953 - accuracy: 0.6805
Epoch 10/10
49/49 [==============================] - 0s 4ms/step - loss: 0.5876 - accuracy: 0.6928


In [185]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 2s 2ms/step - loss: 0.6996 - accuracy: 0.5716


[0.6996147632598877, 0.5715600252151489]

Lo strato di embedding funziona correttamente, ma il risultato è piuttosto scarso, perché ? Perché utilizzare l'embedding senza tener conto della relazione temporale all'interno di una sequenza (come può essere una frase) è una cosa abbastanza inutile. Per tener conto delle informazioni sequenziali dobbiamo usare degli strati ricorrenti.

# Usiamo uno Strato Ricorrente
Le Reti Neurali Ricorrenti (RNN) saranno l'argomento della prossima sezione, qui vediamo semplicemente come varia il risultato utilizzandole insieme all'embedding.

In [186]:
from keras.layers import SimpleRNN

model = Sequential()
#model.add(Embedding(num_words, 50))
model.add(embedding_layer)
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

Come algoritmo di ottimizzazione usiamo l'RMSProp che dovrebbe portare a risultati migliori per le reti ricorrenti, siccome è più lento a convergere rispetto all'ADAM aumentiamo il numero di epoche dell'addestramento a 20.

In [188]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, epochs=20)

Epoch 1/20
49/49 [==============================] - 3s 32ms/step - loss: 0.7241 - accuracy: 0.5050
Epoch 2/20
49/49 [==============================] - 2s 31ms/step - loss: 0.6880 - accuracy: 0.5411
Epoch 3/20
49/49 [==============================] - 2s 31ms/step - loss: 0.6796 - accuracy: 0.5569
Epoch 4/20
49/49 [==============================] - 1s 30ms/step - loss: 0.6718 - accuracy: 0.5724
Epoch 5/20
49/49 [==============================] - 2s 31ms/step - loss: 0.6672 - accuracy: 0.5837
Epoch 6/20
49/49 [==============================] - 2s 31ms/step - loss: 0.6604 - accuracy: 0.5947
Epoch 7/20
49/49 [==============================] - 2s 32ms/step - loss: 0.6581 - accuracy: 0.6008
Epoch 8/20
49/49 [==============================] - 1s 30ms/step - loss: 0.6523 - accuracy: 0.6040
Epoch 9/20
49/49 [==============================] - 2s 31ms/step - loss: 0.6471 - accuracy: 0.6157
Epoch 10/20
49/49 [==============================] - 1s 30ms/step - loss: 0.6429 - accuracy: 0.6172
Epoch 11/

In [187]:
model.evaluate(X_test, y_test)

RuntimeError: ignored